In [1]:
#Before we start, we first run some sql to prepare data.
#1. Run events_lab, events_drug, events_service to generate ‘events_lab.csv’, ‘events_drug.csv’, ‘events_service.csv’
#2. Run patient.sql to generate ‘complete_patient.csv’
#3. Run age.sql to generate age.csv then run charlson.sql to generate charlson.csv in mimiciv-derived database. 

In [2]:
import csv
import pandas as pd

In [3]:
mapping_dict = {
    'Hemoglobin': 'CBC', 'Hematocrit': 'CBC', 'Potassium, Whole Blood': 'Electrolytes',
    'Free Calcium': 'Calcium', 'White Blood Cells': 'CBC', 'MCHC': 'CBC',
    'Red Blood Cells': 'CBC', 'MCH': 'CBC', 'MCV': 'CBC', 'RDW': 'CBC',
    'Hematocrit, Calculated': 'CBC', 'INR(PT)': 'PT', 'Sodium, Whole Blood': 'Electrolytes',
    'Chloride, Whole Blood': 'Electrolytes', 'Calcium, Total': 'Calcium',
    'pH': 'ABG', 'Calculated Total CO2': 'ABG', 'Base Excess': 'ABG',
    'pO2': 'ABG', 'pCO2': 'ABG', 'Potassium': 'Electrolytes', 'Chloride': 'Electrolytes',
    'Sodium': 'Electrolytes', 'Platelet Count': 'CBC','Pneumococcal Vac Polyvalent': 'Pneumococcal Vaccine',
    'PNEUMOcoccal 23-valent polysaccharide vaccine': 'Pneumococcal Vaccine',
    'Phenylephrine': 'PHENYLEPHrine',
    'Oxycodone-Acetaminophen (5mg-325mg)': 'Oxycodone-Acetaminophen',
    'NS': 'Sodium Chloride 0.9%',
    'Metoprolol Tartrate': 'Metoprolol',
    'Metoprolol Succinate XL': 'Metoprolol',
    'LR': 'Lactated Ringers',
    'Insulin Human Regular': 'Insulin',
    'Influenza Virus Vaccine': 'Influenza Vaccine',
    'Influenza Vaccine Quadrivalent': 'Influenza Vaccine',
    'Heparin Sodium': 'Heparin',
    'Epinephrine': 'EPINEPHrine',
    'Docusate Sodium (Liquid)': 'Docusate Sodium',
    'Dextrose 5%': '5% Dextrose',
    'D5W (EXCEL BAG)': '5% Dextrose',
    'D5W': '5% Dextrose',
    'Aspirin EC': 'Aspirin',
    'Albuterol Inhaler': 'Albuterol',
    'Albuterol 0.083% Neb Soln': 'Albuterol',
    'Albumin 5% (25g / 500mL)': 'Albumin',
    'Albumin 5% (12.5g / 250mL)': 'Albumin',
    'Albumin 25% (12.5g / 50mL)': 'Albumin',
    '5% Dextrose (EXCEL BAG)': '5% Dextrose',
    '0.9% Sodium Chloride (Mini Bag Plus)': 'Sodium Chloride 0.9%',
    '0.9% Sodium Chloride': 'Sodium Chloride 0.9%',
    'Furosemide': 'Diuretic',
    'Morphine Sulfate': 'Opioid',
    'HYDROmorphone (Dilaudid)': 'Opioid',
    'Atorvastatin': 'Lipid Lowering',
    'Acetaminophen IV': 'Acetaminophen',
    'Simvastatin': 'Lipid Lowering',
    'TraMADOL (Ultram)': 'Opioid',
    'OxycoDONE (Immediate Release)': 'Opioid',
    'Meperidine': 'Opioid'
}

In [4]:
events_lab = pd.read_csv('events_lab.csv')
events_lab_filtered1 = events_lab[~events_lab['events'].str.contains('Specimen Type', case=False)]
events_lab_filtered1.to_csv('filtered_events_full_lab.csv', index=False)
print(len(events_lab_filtered1))

166935


In [5]:
events_drug = pd.read_csv('events_drug.csv')
events_drug_filtered2 = events_drug.loc[(events_drug['events'] != 'Bag') & (events_drug['events'] != 'Syringe')]
print(len(events_drug_filtered2))

68947


In [6]:
events_service = pd.read_csv('events_service.csv')
print(len(events_service))
combined_df = pd.concat([events_lab_filtered1, events_drug_filtered2, events_service])
combined_df.to_csv('events_full.csv', index=False)

849


In [7]:
print(combined_df.head())
print(len(combined_df))

   patientid                events            starttime
1   10002013           Base Excess  2160-05-18 09:19:00
2   10002013  Calculated Total CO2  2160-05-18 09:19:00
3   10002013          Free Calcium  2160-05-18 09:19:00
4   10002013               Glucose  2160-05-18 09:19:00
5   10002013            Hemoglobin  2160-05-18 09:19:00
236731


In [8]:
# Replace event names with event names in dictionary to combine some events,if the event name doesn't exist in the dictionary, keep the original name
combined_df['events'] = combined_df['events'].replace(mapping_dict)
combined_df.to_csv('events_combined.csv', index=False)  
print(combined_df.head())
print(len(combined_df))

   patientid   events            starttime
1   10002013      ABG  2160-05-18 09:19:00
2   10002013      ABG  2160-05-18 09:19:00
3   10002013  Calcium  2160-05-18 09:19:00
4   10002013  Glucose  2160-05-18 09:19:00
5   10002013      CBC  2160-05-18 09:19:00
236731


In [9]:
combined_df['starttime'] = pd.to_datetime(combined_df['starttime'])
sorted_df = combined_df.sort_values(by=['patientid', 'starttime'], ascending=[True, True])
print(sorted_df.head())
print(len(sorted_df))

   patientid   events            starttime
0   10002013    CSURG  2160-05-18 01:49:32
1   10002013      ABG  2160-05-18 09:19:00
2   10002013      ABG  2160-05-18 09:19:00
3   10002013  Calcium  2160-05-18 09:19:00
4   10002013  Glucose  2160-05-18 09:19:00
236731


In [11]:
df_unique = sorted_df.drop_duplicates()

In [12]:
print(df_unique.head())
print(len(df_unique))

   patientid   events           starttime
0   10002013    CSURG 2160-05-18 01:49:32
1   10002013      ABG 2160-05-18 09:19:00
3   10002013  Calcium 2160-05-18 09:19:00
4   10002013  Glucose 2160-05-18 09:19:00
5   10002013      CBC 2160-05-18 09:19:00
127846


In [13]:
patient_count = df_unique['patientid'].nunique()
print(patient_count)

847


In [14]:
event_count = df_unique['events'].nunique()
print(event_count)

58


In [15]:
df_unique.to_csv('events_final.csv', index=False)  